In [1]:
ls

jinnan.ipynb                       jinnan_round1_testA_20181227.csv  submit.csv
jinnan_round1_submit_20181227.csv  jinnan_round1_train_20181227.csv


In [2]:
import numpy as np 
import pandas as pd 
import lightgbm as lgb
import xgboost as xgb
from sklearn.linear_model import BayesianRidge
from sklearn.linear_model import ARDRegression
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold, RepeatedKFold
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from scipy import sparse
import warnings
import time
import sys
import os
import re
import datetime
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls
from sklearn.metrics import mean_squared_error
from sklearn.metrics import log_loss
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings("ignore")
pd.set_option('display.max_columns',None)
pd.set_option('max_colwidth',100)

In [3]:
train = pd.read_csv('jinnan_round1_train_20181227.csv', encoding = 'gb18030')
test  = pd.read_csv('jinnan_round1_testA_20181227.csv', encoding = 'gb18030')

In [4]:
train.head()

,样本id,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,A11,A12,A13,A14,A15,A16,A17,A18,A19,A20,A21,A22,A23,A24,A25,A26,A27,A28,B1,B2,B3,B4,B5,B6,B7,B8,B9,B10,B11,B12,B13,B14,收率
0,sample_1528,300,NaN,405.0,700,13:30:00,38.0,NaN,NaN,15:30:00,100,16:30:00,102,0.2,17:30:00,103.0,18:30:00,104.0,0.2,300,21:00-21:30,50.0,9.0,5.0,22:00:00,75,22:30:00,70,6:30-7:00,350.0,3.5,3.5,7:00-8:00,8:00:00,65,11:30:00,45.0,11:30-13:00,14:00-15:30,NaN,800.0,0.15,400,0.879
1,sample_1698,300,NaN,405.0,700,14:00:00,29.0,NaN,NaN,16:00:00,101,17:00:00,103,0.2,18:00:00,104.0,19:00:00,105.0,0.2,200,19:00-20:00,50.0,9.0,5.0,20:00:00,80,21:00:00,73,21:00-22:00,320.0,3.5,3.5,22:00-23:00,23:00:00,80,6:00:00,45.0,6:00-7:30,7:30-9:00,9:00-10:00,1200.0,0.15,400,0.902
2,sample_639,300,NaN,405.0,700,14:00:00,29.0,NaN,NaN,16:00:00,102,17:00:00,103,0.2,18:00:00,104.0,19:00:00,105.0,0.2,200,19:00-19:30,50.0,9.0,5.0,20:00:00,79,21:00:00,73,21:00-22:00,320.0,3.5,3.5,22:00-23:00,23:00:00,80,1:00:00,45.0,1:00-2:30,2:30-4:00,4:00-5:00,1200.0,0.15,400,0.936
3,sample_483,300,NaN,405.0,700,1:30:00,38.0,NaN,NaN,3:00:00,100,4:00:00,102,0.2,5:00:00,103.0,6:00:00,104.0,0.2,200,6:30-7:00,50.0,10.0,5.0,7:30:00,70,8:00:00,78,13:30-14:30,290.0,3.5,3.5,14:30-15:30,15:30:00,65,18:00:00,45.0,19:00-20:30,21:30-23:00,NaN,800.0,0.15,400,0.902
4,sample_617,300,NaN,405.0,700,22:00:00,29.0,NaN,NaN,0:00:00,101,1:00:00,103,0.2,2:00:00,104.0,3:00:00,105.0,0.2,200,3:00-4:00,50.0,9.0,5.0,4:00:00,80,5:00:00,73,5:00-6:00,320.0,3.5,3.5,6:00-7:00,7:00:00,80,9:00:00,45.0,9:00-10:30,10:30-12:00,12:00-13:00,1200.0,0.15,420,0.983


In [5]:
# # 删除缺失率超过90%的列
good_cols = list(train.columns)
for col in train.columns:
    rate = train[col].value_counts(normalize=True, dropna=False).values[0]
    if rate > 0.9:
        good_cols.remove(col)

# # 删除异常值
train = train[train['收率']>0.86]
        
train = train[good_cols]
good_cols.remove('收率')
test  = test[good_cols]

In [6]:
# 合并数据集
target = train['收率']
del train['收率']
data = pd.concat([train,test],axis=0,ignore_index=True)
data = data.fillna(-1)

In [7]:
def timeTranSecond(t):
    try:
        t,m,s=t.split(":")
    except:
        if t=='1900/1/9 7:00':
            return 7*3600/3600
        elif t=='1900/1/1 2:30':
            return (2*3600+30*60)/3600
        elif t==-1:
            return -1
        else:
            return 0
    
    try:
        tm = (int(t)*3600+int(m)*60+int(s))/3600
    except:
        return (30*60)/3600
    
    return tm
for f in ['A5','A7','A9','A11','A14','A16','A24','A26','B5','B7']:
    data[f] = data[f].apply(timeTranSecond)

def getDuration(se):
    try:
        sh,sm,eh,em=re.findall(r"\d+\.?\d*",se)
    except:
        if se == -1:
            return -1 
        
    try:
        if int(sh)>int(eh):
            tm = (int(eh)*3600+int(em)*60-int(sm)*60-int(sh)*3600)/3600 + 24
        else:
            tm = (int(eh)*3600+int(em)*60-int(sm)*60-int(sh)*3600)/3600
    except:
        if se=='19:-20:05':
            return 1
        elif se=='15:00-1600':
            return 1
    
    return tm

def get_start(se):
    try:
        sh, sm, eh, em = re.findall(r'\d+\.?d*', se)
    except:
        if se == -1:
            return -1
    try:
        tm = (int(eh) * 3600 + int(em) * 60)/3600
    except:
        if se == '19:-20:05':
            return 19
        elif se == '15:00-1600':
            return 15
    return tm
for f in ['A20','A28','B4','B9','B10','B11']:
    data[f+'_'] = data.apply(lambda df: getDuration(df[f]), axis=1)
for f in ['A20','A28','B4','B9','B10','B11']:
    data[f] = data.apply(lambda df: get_start(df[f]), axis=1)

In [8]:
time_point_cols = ['A5','A7','A9','A11','A14','A16','A20', 'A24','A26','A28','B4','B5','B7','B9','B10','B11']

In [9]:
for col in range(len(time_point_cols)-1):
    _, tmp = divmod(data[time_point_cols[col+1]] - data[time_point_cols[col]], 24)
    data[time_point_cols[col+1]+'-'+time_point_cols[col]] = tmp

In [10]:
cate_columns = [f for f in data.columns if f != '样本id']

In [11]:
def get_float(x):
    try:
        return(float(x))
    except:
        return 70  
for f in cate_columns:
    data[f] = data[f].apply(lambda x: get_float(x))

In [12]:
corr_spy = data[:train.shape[0]]
corr_spy['target'] = list(target)
th = sorted(list(corr_spy.corr().target.abs()),reverse=True)[int(len(corr_spy.columns)*0.88)]
cate_columns = []
for col in corr_spy.corr().columns:
    if corr_spy.corr()['target'][col] >= th:
        cate_columns.append(col)
cate_columns.remove('target')

In [13]:
#label encoder
for f in cate_columns:
    data[f] = data[f].map(dict(zip(data[f].unique(), range(0, data[f].nunique()))))
train = data[:train.shape[0]]
test  = data[train.shape[0]:]

In [14]:
train['target'] = list(target)
train['intTarget'] = pd.cut(train['target'], 6, labels=False)
train = pd.get_dummies(train, columns=['intTarget'])
li = train.columns[-6:]
mean_features = []

In [15]:
for f1 in cate_columns:
    for f2 in li:
        col_name = f1+"_"+f2+'_mean'
        mean_features.append(col_name)
        order_label = train.groupby([f1])[f2].mean()
        for df in [train, test]:
            df[col_name] = df[f1].map(order_label)

train.drop(li, axis=1, inplace=True)

In [16]:
train.drop(['样本id','target'], axis=1, inplace=True)
test = test[train.columns]
X_train = train.values
y_train = target.values
X_test = test.values

In [17]:
param = {'num_leaves': 120,
         'min_data_in_leaf': 30, 
         'objective':'regression',
         'max_depth': -1,
         'learning_rate': 0.02,
         "boosting": "gbdt",
         "feature_fraction": 0.8,
         "bagging_freq": 1,
         "bagging_fraction": 0.8,
         "bagging_seed": 11,
         "metric": 'mse',
         "lambda_l1": 0.1,
         "verbosity": -1}
folds = KFold(n_splits=5, shuffle=True, random_state=666)
oof_lgb = np.zeros(len(train))
predictions_lgb = np.zeros(len(test))

for fold_, (trn_idx, val_idx) in enumerate(folds.split(X_train, y_train)):
    print("fold n°{}".format(fold_+1))
    trn_data = lgb.Dataset(X_train[trn_idx], y_train[trn_idx])
    val_data = lgb.Dataset(X_train[val_idx], y_train[val_idx])

    num_round = 10000
    clf = lgb.train(param, trn_data, num_round, valid_sets = [trn_data, val_data], verbose_eval=200, early_stopping_rounds = 100)
    oof_lgb[val_idx] = clf.predict(X_train[val_idx], num_iteration=clf.best_iteration)
    
    predictions_lgb += clf.predict(X_test, num_iteration=clf.best_iteration) / folds.n_splits

print("CV score: {:<8.8f}".format(mean_squared_error(oof_lgb, target)))

fold n°1
Training until validation scores don't improve for 100 rounds.
[200]	training's l2: 0.000159544	valid_1's l2: 0.000177768
[400]	training's l2: 0.00012653	valid_1's l2: 0.000160618
[600]	training's l2: 0.000113864	valid_1's l2: 0.000156493
Early stopping, best iteration is:
[685]	training's l2: 0.000110336	valid_1's l2: 0.000155963
fold n°2
Training until validation scores don't improve for 100 rounds.
[200]	training's l2: 0.000156141	valid_1's l2: 0.000176597
[400]	training's l2: 0.000121958	valid_1's l2: 0.000166124
Early stopping, best iteration is:
[386]	training's l2: 0.000123185	valid_1's l2: 0.000165993
fold n°3
Training until validation scores don't improve for 100 rounds.
[200]	training's l2: 0.000154086	valid_1's l2: 0.000199695
[400]	training's l2: 0.000121447	valid_1's l2: 0.000176623
[600]	training's l2: 0.000109315	valid_1's l2: 0.000172056
[800]	training's l2: 0.000102288	valid_1's l2: 0.00017033
[1000]	training's l2: 9.72238e-05	valid_1's l2: 0.00016898
[1200]	t

In [18]:
##### xgb
xgb_params = {'n_estimators':120,'eta': 0.005, 'max_depth': 10, 'subsample': 0.9, 'colsample_bytree': 0.8, 
          'objective': 'reg:linear', 'eval_metric': 'rmse', 'silent': True, 'nthread': 4}

folds = KFold(n_splits=5, shuffle=True, random_state=666)
oof_xgb = np.zeros(len(train))
predictions_xgb = np.zeros(len(test))

for fold_, (trn_idx, val_idx) in enumerate(folds.split(X_train, y_train)):
    print("fold n°{}".format(fold_+1))
    trn_data = xgb.DMatrix(X_train[trn_idx], y_train[trn_idx])
    val_data = xgb.DMatrix(X_train[val_idx], y_train[val_idx])

    watchlist = [(trn_data, 'train'), (val_data, 'valid_data')]
    clf = xgb.train(dtrain=trn_data, num_boost_round=20000, evals=watchlist, early_stopping_rounds=200, verbose_eval=100, params=xgb_params)
    oof_xgb[val_idx] = clf.predict(xgb.DMatrix(X_train[val_idx]), ntree_limit=clf.best_ntree_limit)
    predictions_xgb += clf.predict(xgb.DMatrix(X_test), ntree_limit=clf.best_ntree_limit) / folds.n_splits
    
print("CV score: {:<8.8f}".format(mean_squared_error(oof_xgb, target)))

fold n°1
[0]	train-rmse:0.422659	valid_data-rmse:0.423392
Multiple eval metrics have been passed: 'valid_data-rmse' will be used for early stopping.

Will train until valid_data-rmse hasn't improved in 200 rounds.
[100]	train-rmse:0.256808	valid_data-rmse:0.257666
[200]	train-rmse:0.156463	valid_data-rmse:0.157249
[300]	train-rmse:0.095833	valid_data-rmse:0.096683
[400]	train-rmse:0.059334	valid_data-rmse:0.060381
[500]	train-rmse:0.037499	valid_data-rmse:0.038804
[600]	train-rmse:0.02455	valid_data-rmse:0.026305
[700]	train-rmse:0.016975	valid_data-rmse:0.019354
[800]	train-rmse:0.012608	valid_data-rmse:0.015716
[900]	train-rmse:0.010152	valid_data-rmse:0.013926
[1000]	train-rmse:0.008784	valid_data-rmse:0.013097
[1100]	train-rmse:0.008	valid_data-rmse:0.012726
[1200]	train-rmse:0.007511	valid_data-rmse:0.012579
[1300]	train-rmse:0.00717	valid_data-rmse:0.012546
[1400]	train-rmse:0.006916	valid_data-rmse:0.012572
Stopping. Best iteration:
[1283]	train-rmse:0.007219	valid_data-rmse:0.0

In [19]:
# 将lgb和xgb的结果进行stacking
train_stack = np.vstack([oof_lgb,oof_xgb]).transpose()
test_stack = np.vstack([predictions_lgb, predictions_xgb]).transpose()

folds_stack = RepeatedKFold(n_splits=5, n_repeats=2, random_state=666)
oof_stack = np.zeros(train_stack.shape[0])
predictions = np.zeros(test_stack.shape[0])

for fold_, (trn_idx, val_idx) in enumerate(folds_stack.split(train_stack,target)):
    print("fold {}".format(fold_))
    trn_data, trn_y = train_stack[trn_idx], target.iloc[trn_idx].values
    val_data, val_y = train_stack[val_idx], target.iloc[val_idx].values
    
    clf_3 = BayesianRidge()
    clf_3 = ARDRegression()
    #clf_3 = LinearRegression()
    clf_3.fit(trn_data, trn_y)
    
    oof_stack[val_idx] = clf_3.predict(val_data)
    predictions += clf_3.predict(test_stack) / 10
    
mean_squared_error(target.values, oof_stack)

fold 0
fold 1
fold 2
fold 3
fold 4
fold 5
fold 6
fold 7
fold 8
fold 9


0.0001665561300304181

In [22]:
sub_df = pd.read_csv('jinnan_round1_submit_20181227.csv', header=None)
sub_df[1] = predictions
sub_df[1] = sub_df[1].apply(lambda x:round(x, 3))
sub_df.to_csv("submit.csv", index=False, header=None)

In [34]:
(predictions ** 2).sum()/len(predictions)/2

0.4291122545557169